In [ ]:
!pip install selenium==4.2.0

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

In [ ]:
from selenium import webdriver as wd
driver = wd.Chrome(executable_path = 'chromedriver.exe')

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.youtube.com/c/mevpr/videos')

In [ ]:
import time
# 한번 스크롤 후 잠시 멈추는 시간
scroll_pause_time = 1

# 화면의 본문인 body 태그 선택
body = driver.find_element_by_tag_name("body")

# 스크롤 후 화면크기와 스크롤 전의 화면크기가 같을 때까지 스크롤 반복
while True:

    # 현재 화면 길이
    last_height = driver.execute_script(
    "return document.documentElement.scrollHeight"
    )
    
    # end 키를 누름으로써 페이지의 마지막으로 스크롤
    # 유튜브이므로 영상이 밑에 더 있으면 화면이 길어짐
    body.send_keys(Keys.END)
    # 스크롤 후 1초 정지(로딩하는 시간이 있으므로 정지 후 실행)
    time.sleep(scroll_pause_time)
    
    # 스크롤 후의 화면 길이
    new_height = driver.execute_script(
    "return document.documentElement.scrollHeight"
    )
    # 스크롤 후 화면 길이와 스크롤 전 화면 길이가 같다면 멈춤
    if new_height == last_height:
        print("end")
        break


In [ ]:
# youtube 페이지 소스를 page에 대입
page = driver.page_source

In [ ]:
from bs4 import BeautifulSoup
# page에 저장된 태그를 정리해서 리턴
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
# div 태그중 id 속성이 dismissible인 객체 조회(유튜브 영상)
all_videos = soup.find_all(id = "dismissible")

In [ ]:
# 영상 제목을 넣을 리스트를 만듦
title_list = []

for video in all_videos:
    # id 속성이 dismissible인 객체에서 id가 video-title 속성인 객체 조회(제목)
    title = video.find(id = "video-title")
    # 공백을 제거하고 글자수가 0보다 큰 제목이면 리스트에 추가
    if len(title.text.strip())>0:
        title_list.append(title.text)

In [ ]:
# 영상의 조회수를 넣을 리스트를 만듦
view_num_list = []
import re
# 조회수를 포함하는 문자열을 검색하기 위한 객체 생성
view_num_regexp = re.compile(r'조회수')

for video in all_videos:
    
    # 비디오에 포함된 조회수를 리턴
    # span 태그이면서 class 속성이 "inline-metadata-item style-scope ytd-video-meta-block" 인 객체 조회
    view_num = video.find('span',{'class': "inline-metadata-item style-scope ytd-video-meta-block"})
    
    # 문자열에 "조회수" + 문자열이 포함되어 있으면 리스트에 추가
    if view_num_regexp.search(view_num.text):
        view_num_list.append(view_num.text)
        


In [ ]:
# 조회수를 숫자로 변환하여 저장할 리스트 생성
view_number_type_list = []

for view in view_num_list:
    # 조회수를 공백으로 변경(조회수 문자열 삭제)
    view = view.replace("조회수 ", "")

    # 단위가 만이면 view의 마지막 2번째 문자열까지를 실수로 변환 후 10000을 곱해서 리스트에 추가
    if view[-2:] == "만회":
        view_number_type_list.append(int(float(view[:-2])*10000))
    # 단위가 천이면 view의 마지막 2번째 문자열까지를 실수로 변환 후 1000을 곱해서 리스트에 추가                                 
    elif view[-2:] == "천회":
        view_number_type_list.append(int(float(view[:-2])*1000))
    # 단위가 1000 이하이면 view의 마지막 문자열까지를 실수로 변환 후 리스트에 추가
    else:
        view_number_type_list.append(int(view[:-1]))

In [ ]:
    if unit1 == "만회":
        view_number_type_list.append(int(num1*10000))
    elif unit1 == "천회":
        view_number_type_list.append(int(num1*1000))
    else:
        view_number_type_list.append(int(num2))



In [ ]:
# 제목과 조회수를 매칭시킴
dict_youtube = {"title": title_list, "view_num": view_number_type_list}

In [ ]:
import pandas as pd
# 매칭시킨 제목과 조회수를 한 데이터 프레임으로 생성
youtube = pd.DataFrame(dict_youtube)

In [ ]:
# 데이터 프레임을 파일로 저장(줄번호까지 저장되면 read.csv할 때 힘들기 때문에 줄번호는 저장 안 함)
youtube.to_csv("환경부_웹_크롤링.csv", index = False)